# scikit-learnのトレーニング♨

## 性能の評価、チューニング 編
- [分類問題の性能の評価](#分類問題の性能の評価)
- [汎化性能の評価](#汎化性能の評価)
- [チューニング](#チューニング)

## [目次](TableOfContents.ipynb)

## 参考
開発基盤部会 Wiki
- データマイニング（DM）- Python  
https://dotnetdevelopmentinfrastructure.osscons.jp/index.php?%E3%83%87%E3%83%BC%E3%82%BF%E3%83%9E%E3%82%A4%E3%83%8B%E3%83%B3%E3%82%B0%EF%BC%88DM%EF%BC%89-%20Python#y4054d35

## [環境準備](ScikitLearnTraining0.ipynb)

In [ ]:
import io
import requests

import pandas as pd
import numpy as np
import mglearn
import matplotlib.pyplot as plt

from sklearn import datasets                                 # サンプル・データセット
from sklearn.datasets import make_regression                 # 回帰データセット作成
from sklearn.datasets import make_blobs                      # 分類データセット作成
from sklearn.preprocessing import StandardScaler             # 標準化
from sklearn.model_selection import train_test_split         # データ分割

from sklearn.linear_model import LinearRegression            # 線形回帰
from sklearn.preprocessing import PolynomialFeatures         # 多項式回帰の変数変換
from sklearn.linear_model import Ridge                       # 多項式回帰のRidge回帰
from sklearn.linear_model import Lasso                       # 多項式回帰のLASSO回帰
from sklearn.linear_model import ElasticNet                  # 多項式回帰のLASSO回帰
from sklearn.linear_model import Perceptron                  # 単純パーセプトロン線形分類器
from sklearn.linear_model import LogisticRegression          # ロジスティク回帰
from sklearn.svm import SVC                                  # サポートベクターマシン（SVM）分類器
from sklearn.tree import DecisionTreeClassifier              # 決定木（分類木）
from sklearn.ensemble import RandomForestClassifier          # ランダムフォレスト（分類木）
from sklearn.ensemble import GradientBoostingClassifier      # 勾配ブースティング木（分類木）
from sklearn.decomposition import PCA                        # 主成分分析
from sklearn.cluster import KMeans                           # k-means法 クラスタ分析
from sklearn.feature_extraction.text import CountVectorizer  # 自然言語処理ベクトル化
from sklearn.feature_extraction.text import TfidfTransformer # 自然言語ベクトルのTF-IDF計算
from sklearn.decomposition import LatentDirichletAllocation  # 自然言語ベクトルからLDAトピック抽出

from sklearn import metrics                                  # モデル評価
from sklearn.metrics import mean_squared_error as mse        # 精度評価（mse）
from sklearn.metrics import confusion_matrix                 # 混同行列
from sklearn.metrics import silhouette_samples               # シルエット係数
from sklearn.model_selection import cross_val_score          # 交差検証法
from sklearn.model_selection import KFold                    # k分割交差検証法
from sklearn.model_selection import StratifiedKFold          # 層化交差検証法
from sklearn.model_selection import GridSearchCV             # グリッドサーチ

import seaborn as sns                                        # matplotlibラッパ
from mlxtend.plotting import plot_decision_regions           # 決定領域表示関数
from matplotlib import cm                                    # カラーマップ処理

from numpy import linalg as LA                               # 線形代数ライブラリ

In [ ]:
import warnings
warnings.filterwarnings('ignore')

## 分類問題の性能の評価
[分類問題のデータとアルゴリズム](ScikitLearnTraining3.ipynb)を使用。

### データ

In [ ]:
iris = datasets.load_iris()
df_data = pd.DataFrame(iris.data, columns=iris.feature_names)
df_target = pd.DataFrame(iris.target, columns=['species'])
df = pd.concat([df_data, df_target], axis=1)

np_arr=np.array(df)

# データの標準化
ss = StandardScaler()

#### 2値分類

In [ ]:
# PetalLengthCm, PetalWidthCm列の選択
x2=np_arr[50:150, 2:4] 

# Species列の選択
y2=np.array(np_arr[50:150, 4:5],dtype=np.int64) # 要素の型をint64に変換

ss.fit(x2)
x2_std = ss.transform(x2)

# 学習・テストデータの分割
x2_train, x2_test, y2_train, y2_test = train_test_split(x2_std, y2, test_size=0.3, random_state=0)

print(x2.shape)

#### 3値分類

In [ ]:
x3=np_arr[:, 2:4] 
y3=np.array(np_arr[:, 4:5],dtype=np.int64) # 要素の型をint64に変換
ss.fit(x3)
x3_std = ss.transform(x3)
x3_train, x3_test, y3_train, y3_test = train_test_split(x3_std, y3, test_size=0.3, random_state=0)

print(x3.shape)

### アルゴリズム

#### 単純パーセプトロン線形分類器

##### ２値分類

In [ ]:
ppn2 = Perceptron(eta0=0.1) # 学習率 0.1
ppn2.fit(x2_train, np.reshape(y2_train,(-1)))

##### ３値分類

In [ ]:
ppn3 = Perceptron(eta0=0.1) # 学習率 0.1
ppn3.fit(x3_train, np.reshape(y3_train,(-1)))

#### サポートベクターマシン分類器

##### ２値分類

In [ ]:
svc2 = SVC(kernel='rbf', gamma=0.1, C=10)
svc2.fit(x2_train, np.reshape(y2_train,(-1))) 

##### ３値分類

In [ ]:
svc3 = SVC(kernel='rbf', gamma=0.1, C=10)
svc3.fit(x3_train, np.reshape(y3_train,(-1))) 

#### 勾配ブースティング木

##### ２値分類

In [ ]:
tree_gbc2 = GradientBoostingClassifier(random_state=0, max_depth=3, learning_rate=0.1)
tree_gbc2.fit(x2_train, np.reshape(y2_train,(-1)))

##### ３値分類

In [ ]:
tree_gbc3 = GradientBoostingClassifier(random_state=0, max_depth=3, learning_rate=0.1)
tree_gbc3.fit(x3_train, np.reshape(y3_train,(-1)))

### 混同行列

※ [AIプロジェクトのマネジメント > 詳細 > 違い > 色々な難しさ > 混同行列](https://dotnetdevelopmentinfrastructure.osscons.jp/index.php?AI%E3%83%97%E3%83%AD%E3%82%B8%E3%82%A7%E3%82%AF%E3%83%88%E3%81%AE%E3%83%9E%E3%83%8D%E3%82%B8%E3%83%A1%E3%83%B3%E3%83%88#a6b82222)

#### 混同行列表示処理関数

#### 2クラス分類の場合

##### 混同行列表示

###### 単純パーセプトロン線形分類器

In [ ]:
cm_ppn2 = confusion_matrix(y2_test, ppn2.predict(x2_test))
print(cm_ppn2.dtype)
print(cm_ppn2)

###### サポートベクターマシン分類器

In [ ]:
cm_svc2 = confusion_matrix(y2_test, svc2.predict(x2_test))
print(cm_svc2.dtype)
print(cm_svc2)

###### 勾配ブースティング木

In [ ]:
cm_tree2 = confusion_matrix(y2_test, tree_gbc2.predict(x2_test))
print(cm_tree2.dtype)
print(cm_tree2)

##### 混同行列による指標

※ [データマイニング（DM）- CRISP-DM > 詳細 > 評価 > 精度の評価](https://dotnetdevelopmentinfrastructure.osscons.jp/index.php?%E3%83%87%E3%83%BC%E3%82%BF%E3%83%9E%E3%82%A4%E3%83%8B%E3%83%B3%E3%82%B0%EF%BC%88DM%EF%BC%89-%20CRISP-DM#uf759972)

###### accuracy（正解率）

In [ ]:
print("accuracy_score: ", metrics.accuracy_score(y2_test, svc2.predict(x2_test)))

###### precision（感度（再現率））

In [ ]:
print("precision_score: ", metrics.precision_score(y2_test, svc2.predict(x2_test)))

###### recall（精度（適合率））

In [ ]:
print("recall_score: ", metrics.recall_score(y2_test, svc2.predict(x2_test)))

###### f1-score（F値）

In [ ]:
print("f1_score: ", metrics.f1_score(y2_test, svc2.predict(x2_test)))

#### 3クラス分類の場合

##### 混同行列表示

###### 単純パーセプトロン線形分類器

In [ ]:
cm_ppn3 = confusion_matrix(y3_test, ppn3.predict(x3_test))
print(cm_ppn3.dtype)
print(cm_ppn3)

###### サポートベクターマシン分類器

In [ ]:
cm_svc3 = confusion_matrix(y3_test, svc3.predict(x3_test))
print(cm_svc3.dtype)
print(cm_svc3)

###### 勾配ブースティング木

In [ ]:
cm_tree3 = confusion_matrix(y3_test, tree_gbc3.predict(x3_test))
print(cm_tree3.dtype)
print(cm_tree3)

##### ミクロ平均・マクロ平均

###### ミクロ平均
- クラスごとのサンプル数の偏りを調整しない場合の平均の取り方
- 多数派のクラスを重視したいときはミクロ平均を使う。

In [ ]:
# 適合率のミクロ平均
print("precision_score: ", metrics.precision_score(y3_test, svc3.predict(x3_test), average='micro'))
# 再現率のミクロ平均
print("recall_score: ", metrics.recall_score(y3_test, svc3.predict(x3_test), average='micro'))
# F値のミクロ平均
print("f1_score: ", metrics.f1_score(y3_test, svc3.predict(x3_test), average='micro'))

###### マクロ平均
- サンプル数の偏りを調整する場合の平均の取り方
- 全体を均等に評価したいときはマクロ平均を使う。

In [ ]:
# 適合率のマクロ平均
print("precision_score: ", metrics.precision_score(y3_test, svc3.predict(x3_test), average='macro'))
# 再現率のマクロ平均
print("recall_score: ", metrics.recall_score(y3_test, svc3.predict(x3_test), average='macro'))
# F値のマクロ平均
print("f1_score: ", metrics.f1_score(y3_test, svc3.predict(x3_test), average='macro'))

## 汎化性能の評価

### k分割交差検証法
[回帰問題のデータとアルゴリズム](ScikitLearnTraining1.ipynb)を使用。

#### データ

In [ ]:
proxies = { # プロキシ設定
"http":"http://<user_name>:<password>@<proxy_host>:<proxy_port>/",
"https":"https://<user_name>:<password>@<proxy_host>:<proxy_port>/"
}

url = 'https://dotnetdevelopmentinfrastructure.osscons.jp/index.php?plugin=attach&pcmd=open&file=Boston.csv&refer=FrontPage'
res = requests.get(url) # , verify=False, proxies=proxies) # プロキシ
df = pd.read_csv(io.BytesIO(res.content), encoding='utf-8', sep=",")
df=df.drop('Unnamed: 0',axis=1)
df = df.loc[:, ['lstat', 'rm', 'medv']]
x = np.array(df.loc[:, ['lstat', 'rm']])
y = np.array(df.loc[:, ['medv']])

#### アルゴリズム

##### アルゴリズムの初期化

In [ ]:
lr = LinearRegression()

##### k分割交差検証法の設定
- n_splits:  
  - データを分割する通常5〜10個程度の個数。
  - 小さいデータほど数多く分割する必要がある｡
- shuffle:  
  - 分割を行う前にデータをシャッフルするかどうか。
  - Trueにしておいた方が信頼性が上がる。
- random_state:  
  - 乱数生成器の状態。
  - これを固定しておけば､毎回同じ分割結果が得られる。

In [ ]:
kf = KFold(n_splits=5, shuffle=True, random_state=0)

#### 学習

In [ ]:
scores = cross_val_score(lr, x, y, cv=kf)
scores

#### 評価

##### 交差検証精度
- そもそも、scoreはなにか？と言えばR2乗値だった。
- 絶対的な基準でないが0.5以上であれば適合度が高い

###### 交差検証精度の平均

In [ ]:
print(scores.mean()) 

###### 交差検証精度の標準偏差

In [ ]:
print(scores.std())

### 層化k分割交差検証法
[分類問題のデータとアルゴリズム](ScikitLearnTraining3.ipynb)を使用。

#### データ

In [ ]:
iris = datasets.load_iris()
df_data = pd.DataFrame(iris.data, columns=iris.feature_names)
df_target = pd.DataFrame(iris.target, columns=['species'])
df = pd.concat([df_data, df_target], axis=1)
np_arr=np.array(df)
# PetalLengthCm, PetalWidthCm列の選択
x=np_arr[:, 2:4] 
# Species列の選択
y=np.array(np_arr[:, 4:5],dtype=np.int64) # 要素の型をint64に変換

#### アルゴリズム

##### アルゴリズムの初期化

In [ ]:
svc = SVC(kernel='rbf', gamma=0.1, C=10)

##### 層化k分割交差検証法の設定
KFoldと比べるとStratifiedKFoldは、  
クラスラベルを等分するという条件を追加されている。

In [ ]:
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=0)

#### 学習

In [ ]:
scores = cross_val_score(svc, x, y, cv=kf)
scores

#### 評価

###### 交差検証精度の平均

In [ ]:
scores.mean()

###### 交差検証精度の標準偏差

In [ ]:
scores.std()

## チューニング 

### グリッドサーチ
- グリッドサーチはハイパーパラメタのチューニング
- サポートベクターマシン分類機を使用してグリッドサーチ

#### データ

##### XORのデータの作成

In [ ]:
np.random.seed(0)
x_xor = np.random.randn(200, 2)
y_xor = np.logical_xor(x_xor[:, 0] > 0, x_xor[:, 1] > 0)
y_xor = np.where(y_xor, 1, -1)
# データの散布
plt.scatter(x_xor[y_xor == 1, 0], x_xor[y_xor == 1, 1], c='b', marker='x', label='1')
plt.scatter(x_xor[y_xor == -1, 0], x_xor[y_xor == -1, 1], c='r', marker='s', label='-1')
plt.xlim([-3, 3])
plt.ylim([-3, 3])
plt.legend(loc='best') # 右上に凡例を出力
plt.show()

##### ホールド・アウト法による学習・テストデータの分割

In [ ]:
x_xor_train, x_xor_test, y_xor_train, y_xor_test = train_test_split(x_xor, y_xor, test_size=0.3, random_state=0)

#### アルゴリズム
グリッドサーチのアルゴリズムを生成

##### パラメタのグリッドを作成

In [ ]:
param_grid = {
    'C': [0.1, 1.0, 10, 100, 1000, 10000],
    'gamma': [0.001, 0.01, 0.1, 1, 10]}

##### 層化k分割交差検証法の設定

In [ ]:
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=0)

##### アルゴリズムの初期化

In [ ]:
gs_svc = GridSearchCV(SVC(), param_grid, cv=kf)

##### 学習

In [ ]:
gs_svc.fit(X_xor_train, y_xor_train)

##### 評価

###### 結果の出力

In [ ]:
# 精度が最も高かった組み合わせ
print("best_params_: ", gs_svc.best_params_)
# その際のスコア
print("best_score_: ", gs_svc.best_score_)
# データセットの正答率
gs_svc.score(x_xor_test, y_xor_test)

###### 決定境界の出力

In [ ]:
plot_decision_regions(x_xor, y_xor.flatten(), gs_svc)

### 特徴量エンジニアリング

#### 既出の[主成分分析後](ScikitLearnTraining4.ipynb)の[分類](ScikitLearnTraining3.ipynb)など

#### ...